In [2]:
import os
import csv
import xml.etree.ElementTree as ET
from tqdm import tqdm

def parse_xml_to_connections(input_folder, output_csv):
    csv_data = []
    header = ["Linia", "Typ", "Wariant", "Przystanek startowy ID", "Przystanek startowy nazwa", 
              "Przystanek końcowy ID", "Przystanek końcowy nazwa", "Czas przejazdu", "Czasy przejazdu"]

    for root_dir, sub_dirs, files in tqdm(list(os.walk(input_folder))):
        for file in files:
            if file.endswith(".xml"):
                file_path = os.path.join(root_dir, file)
                try:
                    tree = ET.parse(file_path)
                    root = tree.getroot()

                    for linia in root.findall('linia'):

                        linia_nazwa = linia.get('nazwa')
                        linia_typ = linia.get('typ')

                        for wariant in linia.findall('wariant'):
                            wariant_nazwa = wariant.get('nazwa')
                            # print(f"Przetwarzam wariant: {wariant_nazwa}")
                            
                            przystanki = wariant.findall('przystanek')
                            # print(f"Liczba przystanków w wariancie {wariant_nazwa}: {len(przystanki)}")

                            for przystanek in przystanki[:-1]:
                                start_przystanek, koncowy_przystanek = przystanek.findall('czasy/przystanek')[:2]
                                start_id = start_przystanek.get('id')
                                start_nazwa = start_przystanek.get('nazwa')
                                koniec_id = koncowy_przystanek.get('id')
                                koniec_nazwa = koncowy_przystanek.get('nazwa')
                                try:
                                    czas = int(koncowy_przystanek.get('czas', 0))
                                except ValueError:
                                    czas = 0

                                czasy_odjazdu = []
                                dzień = przystanek.find('tabliczka/dzien')
                                if dzień.get('nazwa') != 'w dni robocze':
                                    continue
                                godziny = dzień.findall('godz')
                                for godzina in godziny:
                                    h = int(godzina.get('h'))
                                    for minuta in godzina.findall('min'):
                                        m = int(minuta.get('m'))
                                        czas_odjazdu = h*60 + m
                                        czasy_odjazdu.append(czas_odjazdu)

                                csv_data.append([
                                    linia_nazwa, linia_typ, wariant_nazwa, 
                                    start_id, start_nazwa, 
                                    koniec_id, koniec_nazwa, 
                                    czas, czasy_odjazdu
                                ])
                                
                except Exception as e:
                    print(f"Błąd podczas przetwarzania pliku {file_path}: {e}")

    # Dodanie godziny dojazdu
    for i in range(len(csv_data)-1):
        new_times = []
        if csv_data[i][2] == csv_data[i+1][2] and len(csv_data[i][8]) == len(csv_data[i+1][8]):
            for start1 in csv_data[i][8]:
                min_dist = 1440
                best_start = None
                for start2 in csv_data[i+1][8]:
                    dist = start2 - start1
                    if dist < 0:
                        dist += 1440
                    if dist < min_dist:
                        min_dist = dist
                        best_start = start2
                new_times.append([start1, best_start])
        else:
            new_times = [[start, (start + csv_data[i][7])%1440] for start in csv_data[i][8]]
        csv_data[i][8] = new_times
    csv_data[-1][8] = [[start, (start + csv_data[-1][7])%1440] for start in csv_data[-1][8]]

    with open(output_csv, mode='w', encoding='utf-8', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        writer.writerows(csv_data)
    print(f"Zakończono przetwarzanie. Zapisano {len(csv_data)} połączeń do pliku CSV.")

input_folder = "XML-rozkladyjazdy" 
output_csv = "graf_polaczen2.csv" 
parse_xml_to_connections(input_folder, output_csv)


100%|██████████| 127/127 [00:14<00:00,  9.07it/s]


Zakończono przetwarzanie. Zapisano 14978 połączeń do pliku CSV.
